In [134]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Cargando los datos

In [135]:
indices_marginacion = pd.read_csv("Base_Indice_de_marginacion_municipal_90-15.csv", encoding = "ISO-8859-1", index_col="MUN")

## Visualizar de manera muy general los datos

In [136]:
print(indices_marginacion.shape)
print(list(indices_marginacion.columns))
indices_marginacion.head(20)

(14646, 22)
['CVE_ENT', 'ENT', 'CVE_MUN', 'POB_TOT', 'VP', 'ANALF', 'SPRIM', 'OVSDE', 'OVSEE', 'OVSAE', 'VHAC', 'OVPT', 'PL<5000', 'PO2SM', 'OVSD', 'OVSDSE', 'IM', 'GM', 'IND0A100', 'LUG_NAC', 'LUGAR_EST', 'AÑO']


,CVE_ENT,ENT,CVE_MUN,POB_TOT,VP,ANALF,SPRIM,OVSDE,OVSEE,OVSAE,...,PL<5000,PO2SM,OVSD,OVSDSE,IM,GM,IND0A100,LUG_NAC,LUGAR_EST,AÑO
MUN,,,,,,,,,,,,,,,,,,,,,
Aguascalientes,1,Aguascalientes,1001,877190,-,2.06,9.54,0.31,0.16,0.72,...,8.73,31.13,-,-,-1.676,Muy bajo,-,2408,11,2015
Aguascalientes,1,Aguascalientes,1001,797010,-,2.59,11.48,0.49,0.31,0.77,...,8.73,29.9,-,-,-1.768,Muy bajo,7.69,2409,-,2010
Aguascalientes,1,Aguascalientes,1001,723043,-,3.19,13.61,0.77,0.54,1.54,...,8.21,28.37,-,-,-1.831,Muy bajo,-,2419,11,2005
Aguascalientes,1,Aguascalientes,1001,643419,-,3.86,18.04,-,1.12,0.88,...,7.67,37.24,-,1.5,-1.871,Muy bajo,-,2408,11,2000
Aguascalientes,1,Aguascalientes,1001,582827,121790,4.53,-,-,1.62,1.14,...,-,-,2.12,-,-1.735,Muy bajo,-,2393,-,1995
Aguascalientes,1,Aguascalientes,1001,506274,-,6.05,27.99,6.55,3.64,3,...,11.47,58.36,-,-,-1.833,Muy bajo,-,2341,9,1990
Jesús María,1,Aguascalientes,1005,120405,-,3.26,13.73,0.44,0.37,0.73,...,45.17,33.77,-,-,-1.256,Muy bajo,-,2229,10,2015
Jesús María,1,Aguascalientes,1005,99590,-,3.76,18.21,1.26,1.17,1.43,...,45.17,31.15,-,-,-1.262,Muy bajo,13.411,2202,-,2010
Jesús María,1,Aguascalientes,1005,82623,-,5.33,23.64,1.57,1.4,2.48,...,41.58,36.01,-,-,-1.234,Muy bajo,-,2188,9,2005


In [157]:
grupo_año = indices_marginacion.groupby(['AÑO', 'ENT'])
grupo_año.get_group((2015, 'Sonora')) 

,CVE_ENT,ENT,CVE_MUN,POB_TOT,VP,ANALF,SPRIM,OVSDE,OVSEE,OVSAE,...,PL<5000,PO2SM,OVSD,OVSDSE,IM,GM,IND0A100,LUG_NAC,LUGAR_EST,AÑO
MUN,,,,,,,,,,,,,,,,,,,,,
Hermosillo,26,Sonora,26030,884273,-,1.29,7.79,0.57,0.4,1.58,...,4.13,19.95,-,-,-1.728,Muy bajo,-,2420,71,2015
Cajeme,26,Sonora,26018,433050,-,2.08,9.34,0.63,0.49,1.08,...,10.02,35.81,-,-,-1.478,Muy bajo,-,2331,63,2015
Nogales,26,Sonora,26043,233952,-,0.74,6.65,0.3,0.46,9.71,...,3.52,27.3,-,-,-1.557,Muy bajo,-,2363,67,2015
San Luis Río Colorado,26,Sonora,26055,192739,-,2.97,14.72,0.64,2.25,1.45,...,8.26,30.22,-,-,-1.292,Muy bajo,-,2255,54,2015
Navojoa,26,Sonora,26042,163650,-,2.71,12.72,2.4,1.31,3.06,...,27.83,45.27,-,-,-1.009,Bajo,-,2051,35,2015
Guaymas,26,Sonora,26029,158046,-,2.61,11.87,2.3,1.92,1.92,...,13.69,34.52,-,-,-1.258,Muy bajo,-,2232,51,2015
Caborca,26,Sonora,26017,85631,-,3.29,15.98,0.86,0.72,1.21,...,26.3,31.72,-,-,-1.169,Muy bajo,-,2179,45,2015
Agua Prieta,26,Sonora,26002,82918,-,1.53,10.29,0.23,1.76,1.03,...,2.38,33.26,-,-,-1.455,Muy bajo,-,2316,61,2015
Huatabampo,26,Sonora,26033,80524,-,4.21,17.3,1.95,1.42,6.83,...,61.58,57.63,-,-,-0.542,Bajo,-,1646,13,2015


In [160]:
indices_marginacion.reset_index()
sonora2015 = grupo_año.get_group((2015, 'Sonora'))
sns.barplot(x = sonora2015['ANALF'], y = sonora2015.index)

TypeError: unsupported operand type(s) for /: 'str' and 'int'